In [1]:
!pip install langchain PyMuPDF faiss-cpu sentence-transformers transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 M

In [3]:
!pip install -U langchain-community
import fitz
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer

def extract_from_pdf(pdf_file):
  doc = fitz.open(pdf_file)
  text = ""
  for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text += page.get_text()
  return text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


In [4]:
pdf_file = "./tesla.pdf"
document_text = extract_from_pdf(pdf_file)

In [5]:
print(len(document_text))

881486


In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def chunk_text(text, chunk_size=1000, overlap=100):
  chunks = []
  start = 0
  while start <len(text):
    end = start + chunk_size
    if end > len(text):
      end = len(text)
    chunks.append(text[start:end])
    start += chunk_size - overlap
  return chunks

In [9]:
chunks = chunk_text(document_text)

In [10]:
vector_store = FAISS.from_texts(chunks, embedding_model)

In [11]:
vector_store

In [13]:
vector_store.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [27]:
tokenizer = AutoTokenizer.from_pretrained("crumb/nano-mistral")
model = AutoModelForCausalLM.from_pretrained("crumb/nano-mistral")

tokenizer_config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/340M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [28]:
retriever = vector_store.as_retriever()

In [29]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

In [30]:
from transformers import pipeline

In [31]:
hf_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer, max_new_tokens=200)

The model 'MistralForCausalLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [32]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,chain_type="stuff", retriever=retriever)

In [33]:
def query_rag(question):
  response = rag_pipeline(question)
  return response

In [35]:
question = "What is tesla earnings"
response = query_rag(question)
print(response["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

ders’ equity
 
 
44,704   
 
30,189  
Noncontrolling interests in subsidiaries
 
 
785   
 
826  
Total liabilities and equity
 
$
82,338   
$
62,131  
 
(1)
Prior period results have been adjusted to reflect the three-for-one stock split effected in the form of a stock dividend in August 2022. See Note 
1, Overview, for details. 
 
The accompanying notes are an integral part of these consolidated financial statements.
48
 
Tesla, Inc.
Consolidated Statements of Operations
(in millions, except per share data)
  
 
 
Year Ended December 31,
 
 
 
2022
  
2021
  
2020
 
Revenues
 
   
   
  
Automotive sales
 
$
67,210   
$
44,125   
$
24,604  
Automotive regulatory credits
 
 
1,776   
 
1,465   
 
1,580  
Automotive leasing
 
 
2,476  
  
1,642  
  
1,052  
Total automotive revenues
 
 
71,462   
 
47,232   
 
27,236  
Energ

In [36]:
print(query_rag("how many cars were sold?")["result"])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.


customer.
Services and other revenue consists of non-warranty after-sales vehicle services and parts, paid Supercharging, sales of used vehicles, retail 
merchandise and vehicle insurance revenue.
2022 compared to 2021
Automotive sales revenue increased $23.09 billion, or 52%, in the year ended December 31, 2022 as compared to the year ended December 31, 
2021, primarily due to an increase of 347,024 Model 3 and Model Y deliveries, and an increase of 38,183 Model S and Model X deliveries year over 
year. This was achieved from production ramping of Model Y at Gigafactory Shanghai and the Fremont Factory as well as the start of production at 
Gigafactory Berlin-Brandenburg and Gigafactory Texas in 2022, at a higher combined average selling price from a higher proportion of Model Y sales 
despite a negative impact from the Un